## 149 yyyymmあたりのamount, transaction

In [2]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [3]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
df_hist  = reduce_mem_usage(df_hist )
df_new   = reduce_mem_usage(df_new  )

100%|██████████| 3/3 [00:01<00:00,  1.53it/s]


Mem. usage decreased to  5.58 Mb (48.2% reduction)
Mem. usage decreased to  3.18 Mb (43.8% reduction)
Mem. usage decreased to 1971.22 Mb (40.8% reduction)
Mem. usage decreased to 140.41 Mb (37.5% reduction)


In [5]:
df = df_new
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
# df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df = df[[key, 'month_lag', 'authorized_flag', 'category_1', 'purchase_amount', 'installments', 'diff_days']]
df_new = df.copy()
del df
gc.collect()

df = df_hist
df.sort_values(by=[key, 'purchase_date'], inplace=True)
df['authorized_flag'] =  df['authorized_flag'].map(lambda x: 0 if x=='N' else 1)
df['category_1'] =  df['category_1'].map(lambda x: 0 if x=='N' else 1)
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
# df['yyyymm'] = df['purchase_date'].map(lambda x: str(x)[:7])
df['purchase_date_shift1'] =  df.groupby(key)['purchase_date'].shift(1)
df['diff_days'] =  (df['purchase_date'] - df['purchase_date_shift1']).dt.days
df = df[[key, 'month_lag', 'authorized_flag', 'category_1', 'purchase_amount', 'installments', 'diff_days']]
df_hist = df.copy()
del df
gc.collect()
df_hist.head()

,card_id,month_lag,authorized_flag,category_1,purchase_amount,installments,diff_days
19095896,C_ID_00007093c1,-12,1,0,0.037176,2,NaN
19095775,C_ID_00007093c1,-12,1,0,-0.467415,1,0.0
19095845,C_ID_00007093c1,-12,1,0,-0.572600,1,1.0
19095866,C_ID_00007093c1,-12,1,0,-0.331575,3,3.0
19095808,C_ID_00007093c1,-11,1,0,-0.483944,1,10.0


In [6]:
df_trans = pd.concat([df_hist, df_new], axis=0)
df_trans = reduce_mem_usage(df_trans)
auth1 = df_trans[df_trans['authorized_flag']==1]
auth0 = df_trans[df_trans['authorized_flag']==0]
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)

Mem. usage decreased to 800.17 Mb (44.9% reduction)


### 月を横持ちにして、各月の特徴量にする

In [34]:
feat_cols = ['category_1', 'purchase_amount', 'installments', 'diff_days', 'record']

# auth1
df = auth1.copy()
fname = 'auth1'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'std', 'sum']
aggs['purchase_amount'] = ['mean', 'max', 'min', 'sum']
aggs['diff_days'] = ['mean', 'std']
aggs['installments'] = ['mean', 'max']
aggs['record'] = ['sum']

df['month_lag'] =  df['month_lag'].map(lambda x:"month_lag_" + str(x))
tmp = df.groupby([key, 'month_lag']).agg(aggs)
new_columns = get_new_columns(fname, aggs)
tmp.columns = new_columns

use_cols = tmp.columns
tmp_list = []
for col in use_cols:
    tmp[col] = tmp[col].astype('float64')
    tmp2 = tmp[col].unstack()
    for month in tmp2.columns:
        tmp2.rename(columns={month:f"{month}_{col}"}, inplace=True)
    tmp_list.append(tmp2)
    
df_auth1 = pd.concat(tmp_list, axis=1)
df_auth1.head()

month_lag,month_lag_-1_auth1_category_1_mean,month_lag_-10_auth1_category_1_mean,month_lag_-11_auth1_category_1_mean,month_lag_-12_auth1_category_1_mean,month_lag_-13_auth1_category_1_mean,month_lag_-2_auth1_category_1_mean,month_lag_-3_auth1_category_1_mean,month_lag_-4_auth1_category_1_mean,month_lag_-5_auth1_category_1_mean,month_lag_-6_auth1_category_1_mean,month_lag_-7_auth1_category_1_mean,month_lag_-8_auth1_category_1_mean,month_lag_-9_auth1_category_1_mean,month_lag_0_auth1_category_1_mean,month_lag_1_auth1_category_1_mean,month_lag_2_auth1_category_1_mean,month_lag_-1_auth1_category_1_std,month_lag_-10_auth1_category_1_std,month_lag_-11_auth1_category_1_std,month_lag_-12_auth1_category_1_std,month_lag_-13_auth1_category_1_std,month_lag_-2_auth1_category_1_std,month_lag_-3_auth1_category_1_std,month_lag_-4_auth1_category_1_std,month_lag_-5_auth1_category_1_std,month_lag_-6_auth1_category_1_std,month_lag_-7_auth1_category_1_std,month_lag_-8_auth1_category_1_std,month_lag_-9_auth1_category_1_std,month_lag_0_auth1_category_1_std,month_lag_1_auth1_category_1_std,month_lag_2_auth1_category_1_std,month_lag_-1_auth1_category_1_sum,month_lag_-10_auth1_category_1_sum,month_lag_-11_auth1_category_1_sum,month_lag_-12_auth1_category_1_sum,month_lag_-13_auth1_category_1_sum,month_lag_-2_auth1_category_1_sum,month_lag_-3_auth1_category_1_sum,month_lag_-4_auth1_category_1_sum,month_lag_-5_auth1_category_1_sum,month_lag_-6_auth1_category_1_sum,month_lag_-7_auth1_category_1_sum,month_lag_-8_auth1_category_1_sum,month_lag_-9_auth1_category_1_sum,month_lag_0_auth1_category_1_sum,month_lag_1_auth1_category_1_sum,month_lag_2_auth1_category_1_sum,month_lag_-1_auth1_purchase_amount_mean,month_lag_-10_auth1_purchase_amount_mean,month_lag_-11_auth1_purchase_amount_mean,month_lag_-12_auth1_purchase_amount_mean,month_lag_-13_auth1_purchase_amount_mean,month_lag_-2_auth1_purchase_amount_mean,month_lag_-3_auth1_purchase_amount_mean,month_lag_-4_auth1_purchase_amount_mean,month_lag_-5_auth1_purchase_amount_mean,month_lag_-6_auth1_purchase_amount_mean,month_lag_-7_auth1_purchase_amount_mean,month_lag_-8_auth1_purchase_amount_mean,month_lag_-9_auth1_purchase_amount_mean,month_lag_0_auth1_purchase_amount_mean,month_lag_1_auth1_purchase_amount_mean,month_lag_2_auth1_purchase_amount_mean,month_lag_-1_auth1_purchase_amount_max,month_lag_-10_auth1_purchase_amount_max,month_lag_-11_auth1_purchase_amount_max,month_lag_-12_auth1_purchase_amount_max,month_lag_-13_auth1_purchase_amount_max,month_lag_-2_auth1_purchase_amount_max,month_lag_-3_auth1_purchase_amount_max,month_lag_-4_auth1_purchase_amount_max,month_lag_-5_auth1_purchase_amount_max,month_lag_-6_auth1_purchase_amount_max,month_lag_-7_auth1_purchase_amount_max,month_lag_-8_auth1_purchase_amount_max,month_lag_-9_auth1_purchase_amount_max,month_lag_0_auth1_purchase_amount_max,month_lag_1_auth1_purchase_amount_max,month_lag_2_auth1_purchase_amount_max,month_lag_-1_auth1_purchase_amount_min,month_lag_-10_auth1_purchase_amount_min,month_lag_-11_auth1_purchase_amount_min,month_lag_-12_auth1_purchase_amount_min,month_lag_-13_auth1_purchase_amount_min,month_lag_-2_auth1_purchase_amount_min,month_lag_-3_auth1_purchase_amount_min,month_lag_-4_auth1_purchase_amount_min,month_lag_-5_auth1_purchase_amount_min,month_lag_-6_auth1_purchase_amount_min,month_lag_-7_auth1_purchase_amount_min,month_lag_-8_auth1_purchase_amount_min,month_lag_-9_auth1_purchase_amount_min,month_lag_0_auth1_purchase_amount_min,month_lag_1_auth1_purchase_amount_min,month_lag_2_auth1_purchase_amount_min,month_lag_-1_auth1_purchase_amount_sum,month_lag_-10_auth1_purchase_amount_sum,month_lag_-11_auth1_purchase_amount_sum,month_lag_-12_auth1_purchase_amount_sum,month_lag_-13_auth1_purchase_amount_sum,month_lag_-2_auth1_purchase_amount_sum,month_lag_-3_auth1_purchase_amount_sum,month_lag_-4_auth1_purchase_amount_sum,month_lag_-5_auth1_purchase_amount_sum,month_lag_-6_auth1_purchase_amount_sum,month_lag_-7_auth1_purchase_amount_sum,month_lag_-8_au

In [35]:
feat_cols = ['category_1', 'purchase_amount', 'installments', 'diff_days', 'record']

# auth0
df = auth0.copy()
fname = 'auth0'
df['record'] = 1

aggs = {}
aggs['category_1'] = ['mean', 'std', 'sum']
aggs['purchase_amount'] = ['mean', 'max', 'min', 'sum']
aggs['diff_days'] = ['mean', 'std']
aggs['installments'] = ['mean', 'max']
aggs['record'] = ['sum']

df['month_lag'] =  df['month_lag'].map(lambda x:"month_lag_" + str(x))
tmp = df.groupby([key, 'month_lag']).agg(aggs)
new_columns = get_new_columns(fname, aggs)
tmp.columns = new_columns

use_cols = tmp.columns
tmp_list = []
for col in use_cols:
    tmp[col] = tmp[col].astype('float64')
    tmp2 = tmp[col].unstack()
    for month in tmp2.columns:
        tmp2.rename(columns={month:f"{month}_{col}"}, inplace=True)
    tmp_list.append(tmp2)
    
df_auth0 = pd.concat(tmp_list, axis=1)
df_auth0.head()

month_lag,month_lag_-1_auth0_category_1_mean,month_lag_-10_auth0_category_1_mean,month_lag_-11_auth0_category_1_mean,month_lag_-12_auth0_category_1_mean,month_lag_-13_auth0_category_1_mean,month_lag_-2_auth0_category_1_mean,month_lag_-3_auth0_category_1_mean,month_lag_-4_auth0_category_1_mean,month_lag_-5_auth0_category_1_mean,month_lag_-6_auth0_category_1_mean,month_lag_-7_auth0_category_1_mean,month_lag_-8_auth0_category_1_mean,month_lag_-9_auth0_category_1_mean,month_lag_0_auth0_category_1_mean,month_lag_-1_auth0_category_1_std,month_lag_-10_auth0_category_1_std,month_lag_-11_auth0_category_1_std,month_lag_-12_auth0_category_1_std,month_lag_-13_auth0_category_1_std,month_lag_-2_auth0_category_1_std,month_lag_-3_auth0_category_1_std,month_lag_-4_auth0_category_1_std,month_lag_-5_auth0_category_1_std,month_lag_-6_auth0_category_1_std,month_lag_-7_auth0_category_1_std,month_lag_-8_auth0_category_1_std,month_lag_-9_auth0_category_1_std,month_lag_0_auth0_category_1_std,month_lag_-1_auth0_category_1_sum,month_lag_-10_auth0_category_1_sum,month_lag_-11_auth0_category_1_sum,month_lag_-12_auth0_category_1_sum,month_lag_-13_auth0_category_1_sum,month_lag_-2_auth0_category_1_sum,month_lag_-3_auth0_category_1_sum,month_lag_-4_auth0_category_1_sum,month_lag_-5_auth0_category_1_sum,month_lag_-6_auth0_category_1_sum,month_lag_-7_auth0_category_1_sum,month_lag_-8_auth0_category_1_sum,month_lag_-9_auth0_category_1_sum,month_lag_0_auth0_category_1_sum,month_lag_-1_auth0_purchase_amount_mean,month_lag_-10_auth0_purchase_amount_mean,month_lag_-11_auth0_purchase_amount_mean,month_lag_-12_auth0_purchase_amount_mean,month_lag_-13_auth0_purchase_amount_mean,month_lag_-2_auth0_purchase_amount_mean,month_lag_-3_auth0_purchase_amount_mean,month_lag_-4_auth0_purchase_amount_mean,month_lag_-5_auth0_purchase_amount_mean,month_lag_-6_auth0_purchase_amount_mean,month_lag_-7_auth0_purchase_amount_mean,month_lag_-8_auth0_purchase_amount_mean,month_lag_-9_auth0_purchase_amount_mean,month_lag_0_auth0_purchase_amount_mean,month_lag_-1_auth0_purchase_amount_max,month_lag_-10_auth0_purchase_amount_max,month_lag_-11_auth0_purchase_amount_max,month_lag_-12_auth0_purchase_amount_max,month_lag_-13_auth0_purchase_amount_max,month_lag_-2_auth0_purchase_amount_max,month_lag_-3_auth0_purchase_amount_max,month_lag_-4_auth0_purchase_amount_max,month_lag_-5_auth0_purchase_amount_max,month_lag_-6_auth0_purchase_amount_max,month_lag_-7_auth0_purchase_amount_max,month_lag_-8_auth0_purchase_amount_max,month_lag_-9_auth0_purchase_amount_max,month_lag_0_auth0_purchase_amount_max,month_lag_-1_auth0_purchase_amount_min,month_lag_-10_auth0_purchase_amount_min,month_lag_-11_auth0_purchase_amount_min,month_lag_-12_auth0_purchase_amount_min,month_lag_-13_auth0_purchase_amount_min,month_lag_-2_auth0_purchase_amount_min,month_lag_-3_auth0_purchase_amount_min,month_lag_-4_auth0_purchase_amount_min,month_lag_-5_auth0_purchase_amount_min,month_lag_-6_auth0_purchase_amount_min,month_lag_-7_auth0_purchase_amount_min,month_lag_-8_auth0_purchase_amount_min,month_lag_-9_auth0_purchase_amount_min,month_lag_0_auth0_purchase_amount_min,month_lag_-1_auth0_purchase_amount_sum,month_lag_-10_auth0_purchase_amount_sum,month_lag_-11_auth0_purchase_amount_sum,month_lag_-12_auth0_purchase_amount_sum,month_lag_-13_auth0_purchase_amount_sum,month_lag_-2_auth0_purchase_amount_sum,month_lag_-3_auth0_purchase_amount_sum,month_lag_-4_auth0_purchase_amount_sum,month_lag_-5_auth0_purchase_amount_sum,month_lag_-6_auth0_purchase_amount_sum,month_lag_-7_auth0_purchase_amount_sum,month_lag_-8_auth0_purchase_amount_sum,month_lag_-9_auth0_purchase_amount_sum,month_lag_0_auth0_purchase_amount_sum,month_lag_-1_auth0_diff_days_mean,month_lag_-10_auth0_diff_days_mean,month_lag_-11_auth0_diff_days_mean,month_lag_-12_auth0_diff_days_mean,month_lag_-13_auth0_diff_days_mean,month_lag_-2_auth0_diff_days_mean,month_lag_-3_auth0_diff_days_mean,month_lag_-4_auth0_diff_days_mean,month_lag_-5_auth0_diff_days_mean,month_lag_-6_auth0_

In [36]:
# new
df = df_hist.copy()
fname = 'hist'

aggs = {}
aggs['authorized_flag'] = ['mean', 'sum']

tmp = df.groupby([key, 'month_lag']).agg(aggs)
new_columns = get_new_columns(fname, aggs)
tmp.columns = new_columns

use_cols = tmp.columns
tmp_list = []
for col in use_cols:
    tmp2 = tmp.unstack()
    tmp2 = tmp2[col]
    for month in tmp2.columns:
        tmp2.rename(columns={month:f"month_lag_{month}_{col}"}, inplace=True)
    tmp_list.append(tmp2)
    
df_hist_feat = pd.concat(tmp_list, axis=1)
df_hist_feat.head()

month_lag,month_lag_-13_hist_authorized_flag_mean,month_lag_-12_hist_authorized_flag_mean,month_lag_-11_hist_authorized_flag_mean,month_lag_-10_hist_authorized_flag_mean,month_lag_-9_hist_authorized_flag_mean,month_lag_-8_hist_authorized_flag_mean,month_lag_-7_hist_authorized_flag_mean,month_lag_-6_hist_authorized_flag_mean,month_lag_-5_hist_authorized_flag_mean,month_lag_-4_hist_authorized_flag_mean,month_lag_-3_hist_authorized_flag_mean,month_lag_-2_hist_authorized_flag_mean,month_lag_-1_hist_authorized_flag_mean,month_lag_0_hist_authorized_flag_mean,month_lag_-13_hist_authorized_flag_sum,month_lag_-12_hist_authorized_flag_sum,month_lag_-11_hist_authorized_flag_sum,month_lag_-10_hist_authorized_flag_sum,month_lag_-9_hist_authorized_flag_sum,month_lag_-8_hist_authorized_flag_sum,month_lag_-7_hist_authorized_flag_sum,month_lag_-6_hist_authorized_flag_sum,month_lag_-5_hist_authorized_flag_sum,month_lag_-4_hist_authorized_flag_sum,month_lag_-3_hist_authorized_flag_sum,month_lag_-2_hist_authorized_flag_sum,month_lag_-1_hist_authorized_flag_sum,month_lag_0_hist_authorized_flag_sum
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,NaN,1.0,0.7,0.75,0.6,0.684211,0.866667,0.823529,0.8,0.705882,0.833333,0.916667,0.800000,0.666667,NaN,4.0,7.0,9.0,6.0,13.0,13.0,14.0,4.0,12.0,5.0,11.0,8.0,8.0
C_ID_0001238066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.000000,1.000000,0.945946,1.000000,0.958333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,11.0,23.0,35.0,26.0,23.0
C_ID_0001506ef0,1.0,1.0,1.0,NaN,1.0,0.666667,1.000000,1.000000,1.0,1.000000,1.000000,0.944444,1.000000,0.666667,2.0,5.0,8.0,NaN,1.0,2.0,2.0,2.0,2.0,3.0,7.0,17.0,7.0,4.0
C_ID_0001793786,NaN,NaN,NaN,NaN,1.0,1.000000,0.687500,0.866667,1.0,0.945946,0.862069,0.882353,0.789474,1.000000,NaN,NaN,NaN,NaN,2.0,6.0,11.0,26.0,4.0,35.0,25.0,30.0,30.0,20.0
C_ID_000183fdda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.0,0.960000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,22.0,24.0,9.0,26.0,28.0,28.0


In [37]:
for df in [df_auth1, df_auth0, df_hist_feat]:
    df_train = df_train.merge(df.reset_index(), how='left', on=key)
    df_test = df_test.merge(df.reset_index(), how='left', on=key)
print(df_train.shape)

(201917, 394)


### make ratio feature

In [38]:
fname = '149_mlg'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('month_lag')):continue
    length = len(df_train[col].dropna())
    if length/len(df_train)<0.1:
        print(col)
        continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_test_{col}@', obj=df_test[col].values)

month_lag_-12_auth0_category_1_mean
month_lag_-13_auth0_category_1_mean
month_lag_-10_auth0_category_1_std
month_lag_-11_auth0_category_1_std
month_lag_-12_auth0_category_1_std
month_lag_-13_auth0_category_1_std
month_lag_-8_auth0_category_1_std
month_lag_-9_auth0_category_1_std
month_lag_-12_auth0_category_1_sum
month_lag_-13_auth0_category_1_sum
month_lag_-12_auth0_purchase_amount_mean
month_lag_-13_auth0_purchase_amount_mean
month_lag_-12_auth0_purchase_amount_max
month_lag_-13_auth0_purchase_amount_max
month_lag_-12_auth0_purchase_amount_min
month_lag_-13_auth0_purchase_amount_min
month_lag_-12_auth0_purchase_amount_sum
month_lag_-13_auth0_purchase_amount_sum
month_lag_-12_auth0_diff_days_mean
month_lag_-13_auth0_diff_days_mean
month_lag_-10_auth0_diff_days_std
month_lag_-11_auth0_diff_days_std
month_lag_-12_auth0_diff_days_std
month_lag_-13_auth0_diff_days_std
month_lag_-8_auth0_diff_days_std
month_lag_-9_auth0_diff_days_std
month_lag_-12_auth0_installments_mean
month_lag_-13_auth

In [37]:
import glob

p_list = glob.glob('../features/1_first_valid/*.gz')

for p in p_list:
    if p.count('test'):continue
    tmp = utils.read_pkl_gzip(p)
    if len(tmp)<200000:
        print(len(tmp))